<a href="https://colab.research.google.com/github/patch0000/TwitterAPI_sample/blob/main/unfllloweing_git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Twitterのフォローを外すAPI

相互フォローが外れていた際などに利用する  
https://developer.twitter.com/en/docs/twitter-api/users/follows/api-reference/delete-users-source_id-following

Rate limit	
User rate limit: 50 requests per 15-minute window
Max operations	App limit: 500 successful requests per 24-hour window

In [ ]:
#認証情報を以下に設定する
my_bearer_token = "xxxxx"
my_consumer_key = 'xxxxxx'
my_consumer_secret = 'xxxxxx'
my_access_token ='xxxxx'
my_access_token_secret = 'xxxx'
my_id = 1402247255558561793

import pandas as pd

In [ ]:
#以下にフォローを外したいアカウントのIDを設定する。最大50行まで
my_list_str = """\
000000000000000000000
0000000000000011111"""
my_list=my_list_str.split('\n')
print(my_list)


In [ ]:
#@title
from requests_oauthlib import OAuth1Session
import os
import json

id = my_id
params = {"user.fields": "id,name,created_at,description"}

request_token_url = "https://api.twitter.com/oauth/request_token"
consumer_key = my_consumer_key
consumer_secret = my_consumer_secret
oauth = OAuth1Session(consumer_key, client_secret=consumer_secret)

try:
    fetch_response = oauth.fetch_request_token(request_token_url)
except ValueError:
    print(
        "There may have been an issue with the consumer_key or consumer_secret you entered."
    )

resource_owner_key = fetch_response.get("oauth_token")
resource_owner_secret = fetch_response.get("oauth_token_secret")
print("Got OAuth token: %s" % resource_owner_key)

# Get authorization
base_authorization_url = "https://api.twitter.com/oauth/authorize"
authorization_url = oauth.authorization_url(base_authorization_url)
print("Please go here and authorize: %s" % authorization_url)
verifier = input("Paste the PIN here: ")

# Get the access token
access_token_url = "https://api.twitter.com/oauth/access_token"
oauth = OAuth1Session(
    consumer_key,
    client_secret=consumer_secret,
    resource_owner_key=resource_owner_key,
    resource_owner_secret=resource_owner_secret,
    verifier=verifier,
)
oauth_tokens = oauth.fetch_access_token(access_token_url)

access_token = oauth_tokens["oauth_token"]
access_token_secret = oauth_tokens["oauth_token_secret"]

# Make the request
oauth = OAuth1Session(
    consumer_key,
    client_secret=consumer_secret,
    resource_owner_key=access_token,
    resource_owner_secret=access_token_secret,
)

for target_id in my_list:
  response = oauth.delete(
      "https://api.twitter.com/2/users/{}/following/{}".format(my_id, target_id)
  )
  if response.status_code != 200:
      raise Exception(
          "Request returned an error: {} {}".format(
              response.status_code, response.text)
      )
  #print("Response code: {}".format(response.status_code))
  json_response = response.json()
  #print(json.dumps(json_response, indent=4, sort_keys=True))

print("end")
